In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import os
from tqdm import tqdm
import json

%load_ext autoreload
%autoreload 2

In [ ]:
station_dir = "../all/"  # point this to the directory with all the CSV files for each station, both model and observations
os.makedirs("data", exist_ok=True)


# Gather metadata about each station necessary to perform the analysis
If it hasn't been stored yet, calculate it and store it

In [ ]:
if os.path.exists("data/metadata.json"):
    with open("data/metadata.json") as f:
        metadata = json.loads(f.read())
else:
    metadata = dict()
    start_date = np.datetime64("2018-01-01")  # sometimes there's data from just before jan 1 2018, which we want to exclude
    fs = [s for s in os.listdir(station_dir) if s.endswith(".csv")]
    for i, station_fname in tqdm(enumerate(fs), total=len(fs)):
        if station_fname not in metadata:
            metadata[station_fname] = dict()
        if station_fname.startswith("obs"):
            pathname = os.path.join(station_dir, station_fname)
            df = pd.read_csv(pathname, parse_dates=['ISO8601',], date_parser=lambda x: dt.datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ')).sort_values(by=['ISO8601'])
            df = df[df.ISO8601 >= start_date]
            df = df[df.obstype=='no2']
            if len(df) == 0:
                metadata[station_fname] = dict(obs=station_fname.startswith("obs"), n_data=0, quality=np.nan, lat=np.nan, lon=np.nan, mean=np.nan, std=np.nan, max=np.nan, min=np.nan)
                continue
            metadata[station_fname]["obs"] = True
            metadata[station_fname]["mean"] = df.conc_obs.mean()
            metadata[station_fname]["std"] = df.conc_obs.std()
            metadata[station_fname]["max"] = df.conc_obs.max()
            metadata[station_fname]["min"] = df.conc_obs.min()
            station_name = station_fname[4:-4]

        if station_fname.startswith("model_forecast"):
            pathname = os.path.join(station_dir, station_fname)
            df = pd.read_csv(pathname, parse_dates=['ISO8601',], date_parser=lambda x: dt.datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ')).sort_values(by=['ISO8601'])
            df.ISO8601 -= dt.timedelta(minutes=30)  # the timestamps in mod were misaligned
            df = df[df.ISO8601 >= start_date]
            if len(df) == 0:
                metadata[station_fname] = dict(obs=station_fname.startswith("obs"), n_data=0, quality=np.nan, lat=np.nan, lon=np.nan, mean=np.nan, std=np.nan, max=np.nan, min=np.nan)
                continue
            metadata[station_fname]["obs"] = False
            metadata[station_fname]["mean"] = df.NO2.mean()
            metadata[station_fname]["std"] = df.NO2.std()
            metadata[station_fname]["max"] = df.NO2.max()
            metadata[station_fname]["min"] = df.NO2.min()
            station_name = station_fname[len("model_forecast_"):-4]

        metadata[station_fname]["station"] = station_name
        metadata[station_fname]["n_data"] = len(df)
        metadata[station_fname]["quality"] = len(df) / ((df.ISO8601.iloc[-1] - df.ISO8601.iloc[0]).days * 24 + 23)
        metadata[station_fname]["lat"] = df.iloc[0].lat
        metadata[station_fname]["lon"] = df.iloc[0].lon
        if i % 100 == 0:
            with open("data/metadata.json", "w") as f:
                f.write(json.dumps(metadata))

    with open("data/metadata.json", "w") as f:
        f.write(json.dumps(metadata))



# Assign locations to regions

In [ ]:
quality_thresh = 0.5
obs_thresh = 10
region_centers =  {"birmingham": [[52.472927], [-1.902408]], "manchester": [[53.490899], [-2.234560]], "london": [[51.506722], [-0.178576]],
                    "wuhan": [[30.59], [114.31]], "milan": [[45.46], [9.19]], "losangeles": [[34.05], [-118.24]],
                    "paris": [[48.867190], [2.349779]], "seattle": [[47.578678], [-122.301561]],
                    "madrid": [[40.408597], [-3.698773]], 
                    "berlin": [[52.488120], [13.410967]], "barcelona": [[41.396556], [2.164114]], "santiago": [[-33.463444], [-70.638429]],
                    "beijing": [[39.929069], [116.363086]], "shanghai": [[31.185547], [121.526011]], "tokyo": [[35.676772], [139.827864]],
                    "hongkong": [[22.336816], [114.158563]], "teipei": [[25.017115], [121.538235]], "bangkok": [[13.773331], [100.581897]],
                    "jerusalem": [[31.760589], [35.213318]], "newyork": [[40.683845], [-73.988544]]}

region_names = set(region_centers.keys())
regions = dict()

# assign each station to a region based on euclidean distance from a city center
for region in region_names:
    regions[region] = dict()
    for station_fname in metadata.keys():
        lat = metadata[station_fname]["lat"]
        lon = metadata[station_fname]["lon"]
        if (lat - region_centers[region][0][0])**2 + (lon - region_centers[region][1][0])**2 < 0.5**2:
            regions[region][station_fname] = metadata[station_fname]

[(r, len(regions[r])) for r in regions]

In [ ]:
for region in regions:
    for station_fname in regions[region].keys():
        s = regions[region][station_fname]
        if s["obs"] and s["quality"] > quality_thresh and s["mean"] > obs_thresh:
            station_name = station_fname[4:-4]
            mod_fname = f"model_forecast_{station_name}.csv"

            metadata[mod_fname]["station"] = station_name
            metadata[station_fname]["station"] = station_name
            metadata[mod_fname]["clusters"] = metadata[mod_fname].get("clusters", []) + [region]
            metadata[station_fname]["clusters"] = metadata[station_fname].get("clusters", []) + [region]

for station_fname in metadata:
    if "clusters" not in metadata[station_fname].keys():
        metadata[station_fname]["clusters"] = []
        
for row in metadata.values():
    if any([i in {"madrid", "barcelona"} for i in row["clusters"]]):
        row["clusters"].append("madrid-barcelona")
for row in metadata.values():
    if any([i in {"beijing", "shanghai", "wuhan"} for i in row["clusters"]]):
        row["clusters"].append("beijing-shanghai-wuhan")


with open("data/metadata.json", "w") as f:
    f.write(json.dumps(metadata))

# Use DPK to get forecasts for each station

In [ ]:
import station_forecast

# this will take a while
station_forecast.run(station_dir)

# Anomaly detection for a particular region

In [ ]:
import random
import torch
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
from datetime import datetime
import os
import time
import json

%load_ext autoreload
%autoreload 2

In [ ]:
seed = 633

print("[ Using Seed : ", seed, " ]")

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.cuda.manual_seed(seed)
numpy.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
from dpk.koopman_probabilistic import KoopmanProb
from dpk.model_objs import NormalNLL

import pickle
from scipy.stats import norm
import scipy.stats
from scipy import interpolate
import time
from scipy.spatial.distance import mahalanobis
from sklearn.decomposition import PCA

### Collect the data for the stations in this cluster, preprocess/take log, and put into multidim array

In [ ]:
def get_stations(cluster_id, quality_thresh):
    with open("data/metadata.json") as f:
        metadata = json.loads(f.read())

    temp = []
    for fname in metadata:
        temp.append(metadata[fname])
        temp[-1]["fname"] = fname
    df_meta = pd.DataFrame(temp)
    stations = df_meta.loc[[(cluster_id in r.clusters and r.quality > quality_thresh) for r in df_meta.iloc]].station.unique()
    return stations

In [ ]:
def get_z_scores(x, t, koop):
    params = koop.predict(t, covariates=t.reshape(len(t), 1))
    mean_hat = koop.model_obj.mean(params)
    std_hat = koop.model_obj.std(params)
    z_scores = (x - mean_hat) / std_hat
    return z_scores

def get_contiguous_k_samples(k, series):
    if len(series) < k:
        return np.zeros((0, k))
    k_samples = np.zeros((len(series) - k + 1, k))
    for i in range(0, len(series) - k + 1):
        sample = series[i:i+k]
        k_samples[i, :] = sample.flatten()

    return k_samples

In [ ]:
def remove_outliers(train_ts, train_k_averages, lamb):
    sorted_train_k_averages = sorted(train_k_averages)
    len_tr = len(sorted_train_k_averages)
    IQR = sorted_train_k_averages[3 * len_tr // 4] - sorted_train_k_averages[len_tr // 4]
    lower_thresh, upper_thresh = sorted_train_k_averages[len_tr // 4] - lamb * IQR, sorted_train_k_averages[3 * len_tr // 4] + lamb * IQR

    mask = (lower_thresh < train_k_averages) & (train_k_averages < upper_thresh)
    return train_ts[mask], train_k_averages[mask]

def load_z_scores(data_name):
    param_str = None
    for fname in os.listdir("forecasts"):
        if data_name in fname and fname.startswith("koop"):
            param_str = fname[5:-4]
            print(fname)
    if param_str is None:
        raise FileNotFoundError(data_name)

    koop = pickle.load(open(f"forecasts/koop_{param_str}.pkl", 'rb'))
    x = np.load(f"forecasts/x_{param_str}.npy")
    t = np.load(f"forecasts/t_{param_str}.npy")
    return t, get_z_scores(x, t, koop).flatten()

def get_zeta_series(stations, k=168, lamb=2, show=False):
    # !!! make sure these match the data
    train_start_date = dt.datetime(2018, 1, 1) # dt.datetime(2018, 3, 16)
    train_end_date = dt.datetime(2020, 1, 1)  # covid start date dt.datetime(2020, 3, 16) for Seattle and LA, 1/23 for Wuhan, and 3/9 in Italy
    test_end_date = dt.datetime(2021, 1, 1) # dt.datetime(2020, 5, 16)
    t_min = time.mktime(dt.datetime(2018, 1, 1).timetuple())
    train_start_t = time.mktime(train_start_date.timetuple()) - t_min
    train_end_t = time.mktime(train_end_date.timetuple()) - t_min
    test_end_t = time.mktime(test_end_date.timetuple()) - t_min

    train_k_averages = []
    train_ts = []
    test_k_averages = []
    test_ts = []
    for i, station in enumerate(stations):        
        # load the koopman model and relevant data for station
        obs_t, obs_z_scores = load_z_scores(f"obs_{station}")

        # model_analys
        mod_t, mod_z_scores = load_z_scores(f"model_forecast_{station}")

        train_start = np.nonzero(obs_t >= train_start_t)[0].min()
        train_end = np.nonzero(obs_t >= train_end_t)[0].min()
        test_end = np.nonzero(obs_t <= test_end_t)[0].max()

        f = interpolate.interp1d(mod_t, mod_z_scores)
        aligned_mod_z_scores = f(obs_t)
        zeta_scores = obs_z_scores - aligned_mod_z_scores
        train_k_averages.append(get_contiguous_k_samples(k, zeta_scores[train_start:train_end]).mean(axis=1))
        train_ts.append(get_contiguous_k_samples(k, obs_t[train_start:train_end]).max(axis=1))  # represents the "real" time
        test_k_averages.append(get_contiguous_k_samples(k, zeta_scores[train_end:test_end]).mean(axis=1))
        test_ts.append(get_contiguous_k_samples(k, obs_t[train_end:test_end]).max(axis=1))

        # remove outliers from training data
        train_ts[-1], train_k_averages[-1] = remove_outliers(train_ts[-1], train_k_averages[-1], lamb=lamb)
        
    if show:
        plt.figure(dpi=500, figsize=(15, 15))
        for i in range(min(len(stations), 9)):
            plt.subplot(3, 3, i+1)
            plt.hist(train_k_averages[i], bins=40, density=True, color="tab:blue", alpha=0.5, label="train $\\bar z$")
            # plt.hist(control_k_averages, bins=40, density=True, color="tab:orange", alpha=0.5, label="hindcast $\\bar z$")
            plt.hist(test_k_averages[i], bins=40, density=True, color="tab:orange", alpha=0.5, label="covid $\\bar z$")
            plt.title(f"{stations[i]}, k={k}")
            plt.xlabel("sample mean $\zeta$")
            l = np.linspace(min(train_k_averages[i]), max(train_k_averages[i]))
            # if consecutive z-scores were iid, we would expect this distribution to have std = 1/sqrt(k)
            plt.plot(l, norm.pdf(l, loc=np.mean(train_k_averages[i]), scale=np.std(train_k_averages[i])), label="normal fit")
        plt.legend()
        plt.tight_layout()
        plt.show()
        
    good_station_idxs = list(range(len(stations)))
    good_stations = stations[good_station_idxs]
    # run PCA on these, after time aligning them
    all_train_ts = np.unique(np.concatenate(np.array(train_ts, dtype=object)[good_station_idxs])).astype(np.float64)
    all_test_ts = np.unique(np.concatenate(np.array(test_ts, dtype=object)[good_station_idxs])).astype(np.float64)

    train_zeta_avg_mat = np.full((len(all_train_ts), len(stations)), fill_value=np.nan)  # [t, dim]
    test_zeta_avg_mat = np.full((len(all_test_ts), len(stations)), fill_value=np.nan)  # [t, dim]

    for i, station in zip(good_station_idxs, good_stations):
        # TODO deal with train_k_averages being empty. You can probably just pass on error
        interp = interpolate.interp1d(train_ts[i], train_k_averages[i], bounds_error=False, fill_value=np.nanmean(train_k_averages[i]))
        train_zeta_avg_mat[:, i] = interp(all_train_ts)

        interp = interpolate.interp1d(test_ts[i], test_k_averages[i], bounds_error=False, fill_value=np.nanmean(test_k_averages[i]))
        test_zeta_avg_mat[:, i] = interp(all_test_ts)

    train_zeta_avg_mat = train_zeta_avg_mat[:, good_station_idxs]
    test_zeta_avg_mat = test_zeta_avg_mat[:, good_station_idxs]
    return all_train_ts, all_test_ts, train_zeta_avg_mat, test_zeta_avg_mat, train_end_t

In [ ]:
def get_nd_cdf(n, radial_density=norm):
    """returns a function that, given a radius r, returns the probability
    of an observation being as close or closer to the origin in n dimensional
    space than r, assuming observations are drawn from a distribution centered
    at the origin and all of whose radial cross sections have density function `radial_density`"""
    stop = 50
    num = 1000
    while True:
        radii = np.linspace(0, stop, num)
        cdf = np.zeros_like(radii)
        for i, r in enumerate((radii[1:] + radii[:-1]) / 2):
            dr = radii[i + 1] - radii[i]
            shell = radial_density.pdf(r) * r**(n - 1)
            cdf[i + 1] = shell * dr + cdf[i]
        cdf /= max(cdf)

        radial_pdf = cdf[1:] - cdf[:-1]
        if radial_pdf[-1] < 0.001 * max(radial_pdf):
            break
        else:
            stop *= 2
            num *= 2
    return interpolate.interp1d(radii, cdf, bounds_error=True)

def l2_norm(vec, axis=None):
        return np.sqrt(np.sum(vec**2, axis=axis))

In [ ]:
def get_mahalanobis_dist(all_train_ts, all_test_ts, train_zeta_avg_mat, test_zeta_avg_mat, train_end_t, explained_var_thresh=0.8, show=False):
    pca = PCA()
    pca.fit(train_zeta_avg_mat)
    cumul_exp_var = [0] + [sum(pca.explained_variance_ratio_[:i + 1]) for i in range(len(pca.explained_variance_ratio_))]
    if show:
        plt.figure(dpi=500, figsize=(4, 4))
        plt.plot(cumul_exp_var, marker=".")
        plt.xlabel("rank")
        plt.ylabel("explained variance")
        plt.show()
    train_proj = pca.transform(train_zeta_avg_mat)
    test_proj = pca.transform(test_zeta_avg_mat)
    if show:
        plt.figure(dpi=500)
        plt.scatter(train_proj[:, 0], train_proj[:, 1], s=1, c=all_train_ts)
        axes = plt.gca()
        axes.set_aspect("equal")
        plt.xlabel("PC1")
        plt.ylabel("PC2")
        plt.title("training data projection")
        plt.figure(dpi=500, figsize=(11, 11))
        plt.show()
    # norm_means = [] this is just 0
    norm_stds = []
    for i in range(train_proj.shape[1]):
        norm_stds.append(np.std(train_proj[:, i]))

    if show:
        plt.figure(dpi=500, figsize=(8, 20))
        for i in range(min(train_proj.shape[1], 9)):
            plt.subplot(3, 3, i+1)
            # norm_means.append(np.mean(train_proj[:, i]))
            norm_crit_l = norm.ppf(alpha, loc=0, scale=norm_stds[i])
            norm_crit_u = norm.ppf(1-alpha, loc=0, scale=norm_stds[i])
            plt.hist(train_proj[:, i], bins=40, density=True, color="tab:blue", alpha=0.5, label="train $\\bar z$")
            # plt.hist(control_k_averages, bins=40, density=True, color="tab:orange", alpha=0.5, label="hindcast $\\bar z$")
            plt.hist(test_proj[:, i], bins=40, density=True, color="tab:orange", alpha=0.5, label="covid $\\bar z$")
            plt.axvline(norm_crit_l, color="gray", label="$\\bar z^*$")
            plt.axvline(norm_crit_u, color="gray")
            plt.title(f"cluster {cluster_id}, PC {i + 1}, k={k}")
            plt.xlabel("sample mean zeta-score")
            l = np.linspace(min(train_proj[:, i]), max(train_proj[:, i]))
            # if consecutive z-scores were iid, we would expect this distribution to have std = 1/sqrt(k)
            plt.plot(l, norm.pdf(l, loc=0, scale=norm_stds[i]), label="normal fit")
        plt.legend()
        plt.tight_layout()
        plt.show()

        plt.figure(dpi=500, figsize=(11, 22))
        for i in range(train_proj.shape[1]):    
            plt.subplot(6, 3, i+1)
            plt.plot(all_train_ts, train_proj[:, i], label="train  $\\bar z$", color="tab:blue")
            plt.plot(all_test_ts, test_proj[:, i], label="test  $\\bar z$", color="tab:orange")
            plt.ylim([-4, 4])
            plt.axvline(train_end_t, color="gray")
            plt.ylabel("$\\bar \zeta$")
            plt.xlabel("t")
            
        plt.legend()
        plt.tight_layout()
        plt.show()

    mean = np.mean(train_zeta_avg_mat, axis=0)
    err = train_zeta_avg_mat - mean
    cov = err.T @ err / (err.shape[0] - 1)
    icov = np.linalg.inv(cov)
    train_mahalanobis_dist = np.empty(len(train_zeta_avg_mat))
    for i in range(len(train_zeta_avg_mat)):
        train_mahalanobis_dist[i] = mahalanobis(train_zeta_avg_mat[i, :], mean, icov)
    test_mahalanobis_dist = np.empty(len(test_zeta_avg_mat))
    for i in range(len(test_zeta_avg_mat)):
        test_mahalanobis_dist[i] = mahalanobis(test_zeta_avg_mat[i, :], mean, icov)

    num_PCs = np.nonzero(np.array(cumul_exp_var) >= explained_var_thresh)[0].min()
    train_mahalanobis_dist = l2_norm(np.array([train_proj[:, i] / norm_stds[i] for i in range(num_PCs)]), axis=0)
    test_mahalanobis_dist = l2_norm(np.array([test_proj[:, i] / norm_stds[i] for i in range(num_PCs)]), axis=0)
    print(num_PCs, "PCs")
    return train_mahalanobis_dist, test_mahalanobis_dist, num_PCs


In [ ]:
lockdown_dates =  {"birmingham": dt.datetime(2020, 3, 20), "manchester": dt.datetime(2020, 3, 20), "london": dt.datetime(2020, 3, 20),
                    "losangeles": dt.datetime(2020, 3, 16), "seattle": dt.datetime(2020, 3, 16), "wuhan": dt.datetime(2020, 1, 23),
                    "paris": dt.datetime(2020, 3, 17), "milan": dt.datetime(2020, 3, 9),
                    "madrid": dt.datetime(2020, 3, 13), 
                    "berlin": dt.datetime(2020, 3, 19), "barcelona": dt.datetime(2020, 3, 13), "santiago": dt.datetime(2020, 3, 26),
                    "tokyo": dt.datetime(2020, 4, 10), # this is very fuzzy
                    "hongkong": dt.datetime(2020, 1, 25), "teipei": dt.datetime(2021, 5, 15), "bangkok": dt.datetime(2020, 3, 21),
                    "jerusalem": dt.datetime(2020, 3, 15), "newyork": dt.datetime(2020, 3, 22)}  # beijing and shanghai are unclear

In [ ]:
cluster_id = "shanghai"
alpha=0.001
k=168
explained_var_thresh = 0.9
quality_thresh=0.5
stations = get_stations(cluster_id, quality_thresh=quality_thresh)
all_train_ts, all_test_ts, train_zeta_avg_mat, test_zeta_avg_mat, train_end_t = get_zeta_series(stations, k=k, show=False)

## Hierarchical anomaly detection figure

In [ ]:
alpha=0.001
k=168
quality_thresh=0.5
explained_var_thresh=0.9

region_id = "madrid-barcelona"
city_ids = ["madrid", "barcelona"]
all_stations = list(get_stations(region_id, quality_thresh=quality_thresh))
regions = [region_id] + city_ids + all_stations
region_stations = {station: [station] for station in all_stations}
region_stations[region_id] = list(get_stations(region_id, quality_thresh=quality_thresh))
for cid in city_ids:
    region_stations[cid] = list(get_stations(cid, quality_thresh=quality_thresh))

region_train_ts, region_test_ts, region_train_zeta_avg_mat, region_test_zeta_avg_mat, region_train_probs, region_test_probs = dict(), dict(), dict(), dict(), dict(), dict()
for region in region_stations:
    stations = np.array(region_stations[region], dtype='object')

    region_train_ts[region], region_test_ts[region], region_train_zeta_avg_mat[region], region_test_zeta_avg_mat[region], train_end_t = get_zeta_series(stations, k=k, show=False)
    train_mahalanobis_dist, test_mahalanobis_dist, num_PCs = get_mahalanobis_dist(region_train_ts[region], region_test_ts[region],
            region_train_zeta_avg_mat[region], region_test_zeta_avg_mat[region], train_end_t, explained_var_thresh=explained_var_thresh, show=False)
    cdf = get_nd_cdf(n=num_PCs, radial_density=norm)
    region_train_probs[region], region_test_probs[region] = (1 - cdf(abs(train_mahalanobis_dist))), (1 - cdf(abs(test_mahalanobis_dist)))

In [ ]:
import matplotlib
import matplotlib.colors as mcolors
colors = list("bgrcmyk") + list(mcolors.TABLEAU_COLORS.keys())

matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
matplotlib.rc('font', **font)
plt.figure(dpi=500, figsize=(12, 3))
t_scale = 24 * 60 * 60
lt = time.mktime(lockdown_dates["barcelona"].timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
plt.fill_between([lt / t_scale, lt / t_scale + 100], [0, 0], [11, 11], color="y", alpha=0.2, label="lockdown")
plt.fill_between([0, 365 * 2], [0, 0], [11, 11], color="c", alpha=0.2, label="train")
plt.fill_between([365 * 2, lt / t_scale], [0, 0], [11, 11], color="m", alpha=0.2, label="pre-lockdown")
i=0
for region in list(np.random.choice(all_stations, 20, replace=False)) + city_ids:
    plt.plot(region_train_ts[region] / t_scale, region_train_probs[region], linewidth=1, color="grey", alpha=0.35)
    plt.plot(region_test_ts[region] / t_scale, region_test_probs[region], linewidth=1, color="grey", alpha=0.35)
    i = (i + 1) % len(colors)
for region in [region_id]:
    plt.plot(region_train_ts[region] / t_scale, region_train_probs[region], linewidth=2, color="b", label="Spain")
    plt.plot(region_test_ts[region] / t_scale, region_test_probs[region], linewidth=2, color="b")
    i = (i + 1) % len(colors)
plt.xlim([-20, 870])
plt.semilogy()
plt.gca().set_ylim(top=9)
plt.legend(loc="lower left", bbox_to_anchor=[0, 0], ncol=2)
plt.xlabel("days since Jan 1 2018")
plt.ylabel("p-value")
plt.axhline(alpha, color="gray", linestyle="--", label="$\\alpha$")
plt.savefig(f"hierarchy_{region_id}.svg", format="svg")
plt.show()

In [ ]:
plt.figure(dpi=100, figsize=(18, 4))
n_bins = 30
bins = np.array([0] + [10.0**i for i in np.linspace(-12, 0, n_bins)])
# bins = np.linspace(0, 1, n_bins + 1)
titles = ["Spain", "Madrid", "Barcelona"]
for i, region in enumerate([region_id] + city_ids): # + list(np.random.choice(all_stations, 3, replace=False))):
    start_idx = np.nonzero(region_test_ts[region] >= lt)[0].min()
    end_idx = np.nonzero(region_test_ts[region] >= lt + 80 * t_scale)[0].min()
    plt.subplot(1, 4, i + 1)
    plt.title(titles[i])
    prelock, lock = region_test_probs[region][:start_idx - k], region_test_probs[region][start_idx:end_idx]
    plt.hist(prelock, label="pre-lockdown", color="m", alpha=0.4, bins=bins, density=True) #, bins=[0, 0.001, 0.01, 0.1, 0.5, 1])
    plt.hist(lock, label="lockdown", color="y", alpha=0.4, bins=bins, density=True) #, bins=[0, 0.001, 0.01, 0.1, 0.5, 1]
    print(region, np.all(lock > alpha))
    print(region, np.any(prelock < alpha))
    # plt.ylim([0, 10])
    plt.semilogy()
    plt.semilogx()
plt.subplot(1, 4, 4)
plt.title("individual station (avg)")
pre_lockdown_counts = np.zeros(n_bins)
lockdown_counts = np.zeros(n_bins)
false_pos = 0
false_neg = 0
for i, region in enumerate(all_stations):
    start_idx = np.nonzero(region_test_ts[region] >= lt)[0].min()
    end_idx = np.nonzero(region_test_ts[region] >= lt + 80 * t_scale)[0].min()
    prelock, lock = region_test_probs[region][:start_idx - k], region_test_probs[region][start_idx:end_idx]
    pre_lockdown_counts += np.histogram(prelock, bins=bins, density=True)[0]
    lockdown_counts += np.histogram(lock, bins=bins, density=True)[0]
    false_neg += np.all(lock > alpha)
    false_pos += np.any(prelock < alpha)
    # plt.ylim([0, 10])
    plt.xlim([10**-20, 1])
    plt.semilogy()
    plt.semilogx()
pre_lockdown_counts /= len(all_stations)
lockdown_counts /= len(all_stations)
plt.bar(bins[:-1], pre_lockdown_counts, align="edge", label="pre-lockdown", color="m", alpha=0.4, width=bins[1:] - bins[:-1])
plt.bar(bins[:-1], lockdown_counts, align="edge", label="lockdown", color="y", alpha=0.4, width=bins[1:] - bins[:-1])
plt.xlabel("p-value")
plt.ylabel("probability density")
plt.legend(loc="upper right", bbox_to_anchor=[1, 1])
plt.tight_layout()
plt.savefig(f"hierarchy_hists_{region_id}.svg", format="svg")
plt.show()

## Individual station DPK

In [ ]:
station =  "EEA_IT_IT1251A" # "Station0004349" "Station0004701" "Station0004349"  # Seattle  "Station0001766" # LA
data_name = f"obs_{station}"
for fname in os.listdir("forecasts"):
    if data_name in fname and fname.startswith("koop"):
        param_str = fname[5:-4]
        print(fname)

obs_koop = pickle.load(open(f"forecasts/koop_{param_str}.pkl", 'rb'))
obs_x = np.load(f"forecasts/x_{param_str}.npy")
obs_t = np.load(f"forecasts/t_{param_str}.npy")

train_start_date = dt.datetime(2018, 1, 1) # dt.datetime(2018, 3, 16)
train_end_date = dt.datetime(2020, 1, 1)  # covid start date dt.datetime(2020, 3, 16) for Seattle and LA, 1/23 for Wuhan, and 3/9 in Italy
test_end_date = dt.datetime(2021, 1, 1) # dt.datetime(2020, 5, 16)
t_min = time.mktime(dt.datetime(2018, 1, 1).timetuple())
train_start_t = time.mktime(train_start_date.timetuple()) - t_min
train_end_t = time.mktime(train_end_date.timetuple()) - t_min
test_end_t = time.mktime(test_end_date.timetuple()) - t_min

train_start = np.nonzero(obs_t >= train_start_t)[0].min()
train_end = np.nonzero(obs_t >= train_end_t)[0].min()
test_end = np.nonzero(obs_t <= test_end_t)[0].max()

params = obs_koop.predict(obs_t, covariates=obs_t.reshape(len(obs_t), 1))
mean_hat = obs_koop.model_obj.mean(params)
std_hat = obs_koop.model_obj.std(params)

In [ ]:
from scipy.stats import norm

CIs = dict()
percentiles = [0.1, 0.2]
for percentile in percentiles:
    CIs[percentile] = (norm.ppf(percentile, loc=mean_hat.flatten(), scale=std_hat.flatten()), norm.ppf(1 - percentile, loc=mean_hat.flatten(), scale=std_hat.flatten()))

In [ ]:
plt.figure(dpi=500, figsize=(7.5, 2.5))
t_scale = 60 * 60 * 24
plt.plot(obs_t / t_scale - 730, obs_x, c="k", linewidth=0.5, label="observation")
for i, percentile in enumerate(percentiles):
    lo, hi = CIs[percentile]
    label = None if i != 0 else "DPK"
    plt.fill_between(obs_t / t_scale - 730, lo, hi, alpha=0.2, facecolor="b", linewidth=0, label=label)
plt.xlabel("days since Jan 1 2020")
plt.ylabel("$\log[$NO$_2]$")
plt.xlim([45, 100])
plt.ylim([1, 7])
plt.fill_between([68, 200], [0, 0], [10, 10], color="yellow", alpha=0.2, label="lockdown")
plt.legend()
plt.tight_layout()
plt.savefig("madrid_DPK.pdf", format="pdf")
plt.show()

### Parameter sweep over k

In [ ]:
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score, fbeta_score
# F_\beta "measures the effectiveness of retrieval with respect to a user who attaches \beta times as much importance to recall as precision"
# If the class ratio was 50/50 I would assign ~2x as much weight to recall, but since the class ratio is really more like 99/1 I don't want to be spammed with false positives

In [ ]:
lockdown_dates =  {"birmingham": dt.datetime(2020, 3, 20), "manchester": dt.datetime(2020, 3, 20), "london": dt.datetime(2020, 3, 20),
                    "losangeles": dt.datetime(2020, 3, 16), "seattle": dt.datetime(2020, 3, 16), "wuhan": dt.datetime(2020, 1, 23),
                    "paris": dt.datetime(2020, 3, 17), "milan": dt.datetime(2020, 3, 9),
                    "madrid": dt.datetime(2020, 3, 13), 
                    "berlin": dt.datetime(2020, 3, 19), "barcelona": dt.datetime(2020, 3, 13), "santiago": dt.datetime(2020, 3, 26),
                    "beijing": dt.datetime(2020, 1, 1), "shanghai": dt.datetime(2020, 1, 2), "tokyo": dt.datetime(2020, 4, 10), # this is very fuzzy
                    "hongkong": dt.datetime(2020, 1, 25), "teipei": dt.datetime(2021, 5, 15), "bangkok": dt.datetime(2020, 3, 21),
                    "jerusalem": dt.datetime(2020, 3, 15), "newyork": dt.datetime(2020, 3, 22)}  # beijing and shanghai are unclear

In [ ]:
quality_thresh = 0.5
beta = 1/6  # I want 6x more precision than recall
n_regions = len(lockdown_dates)
sweep_regions = np.random.choice(list(lockdown_dates), n_regions, replace=False)
sweep_stations = {region: get_stations(region, quality_thresh=quality_thresh) for region in sweep_regions}

In [ ]:
alphas = [0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]
k_values = [1, 6, 24, 48, 72, 120, 168, 336]
explained_var_thresh_values = [0.4, 0.6, 0.7, 0.8, 0.9, 0.95, 0.999]
lag_mats, F1_mats = {r: np.full((len(explained_var_thresh_values), len(k_values), len(alphas)), np.nan) for r in sweep_regions}, {r: np.full((len(explained_var_thresh_values), len(k_values), len(alphas)), np.nan) for r in sweep_regions}
all_train_ts, all_test_ts, all_train_zeta_avg_mat, all_test_zeta_avg_mat, all_train_probs, all_test_probs = dict(), dict(), dict(), dict(), dict(), dict()
for i, evt in tqdm(list(enumerate(explained_var_thresh_values)), total=len(explained_var_thresh_values)):
    all_train_ts[evt], all_test_ts[evt], all_train_zeta_avg_mat[evt], all_test_zeta_avg_mat[evt], all_train_probs[evt], all_test_probs[evt] = dict(), dict(), dict(), dict(), dict(), dict()
    for j, k in enumerate(k_values):
        all_train_ts[evt][k], all_test_ts[evt][k], all_train_zeta_avg_mat[evt][k], all_test_zeta_avg_mat[evt][k], all_train_probs[evt][k], all_test_probs[evt][k] = dict(), dict(), dict(), dict(), dict(), dict()
        for region in sweep_regions:
            try:
                stations = np.array(sweep_stations[region], dtype='object')

                all_train_ts[evt][k][region], all_test_ts[evt][k][region], all_train_zeta_avg_mat[evt][k][region], all_test_zeta_avg_mat[evt][k][region], all_train_end_t = get_zeta_series(stations, k=k, show=False)
                train_mahalanobis_dist, test_mahalanobis_dist, num_PCs = get_mahalanobis_dist(all_train_ts[evt][k][region], all_test_ts[evt][k][region],
                        all_train_zeta_avg_mat[evt][k][region], all_test_zeta_avg_mat[evt][k][region], train_end_t, explained_var_thresh=evt, show=False)
                cdf = get_nd_cdf(n=num_PCs, radial_density=norm)
                all_train_probs[evt][k][region], all_test_probs[evt][k][region] = (1 - cdf(abs(train_mahalanobis_dist))), (1 - cdf(abs(test_mahalanobis_dist)))

                lt = time.mktime(lockdown_dates[region].timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
                tt = all_test_ts[evt][k][region]
                control = all_test_probs[evt][k][region][tt < lt]
                test = all_test_probs[evt][k][region][(lt + k <= tt) & (tt < lt + 50 * 24 * 60 * 60)]
                for l, alpha in enumerate(alphas):
                    is_anomaly_control = control < alpha
                    is_anomaly_test = test < alpha
                    lab = list(np.full(len(control), False)) + list(np.full(len(test), True))
                    F1_mats[region][i, j, l] = fbeta_score(lab, list(is_anomaly_control) + list(is_anomaly_test), beta=beta, pos_label=True, average="binary")
                    anomaly_ts = all_test_ts[evt][k][region][(lt + k <= tt) & (tt < lt + 50 * 24 * 60 * 60) & (all_test_probs[evt][k][region] < alpha)]
                    lag_mats[region][i, j, l] = anomaly_ts[0] - lt if len(anomaly_ts) > 0 else np.inf
            except Exception as e:
                print("\n\n\nException:", e)

In [ ]:

with open("all_train_ts.pkl", "wb") as f:
    pickle.dump(pd.DataFrame(all_train_ts), f)
with open("all_test_ts.pkl", "wb") as f:
    pickle.dump(pd.DataFrame(all_test_ts), f)
with open("all_train_zeta_avg_mat.pkl", "wb") as f:
    pickle.dump(pd.DataFrame(all_train_zeta_avg_mat), f)
with open("all_test_zeta_avg_mat.pkl", "wb") as f:
    pickle.dump(pd.DataFrame(all_test_zeta_avg_mat), f)
with open("all_train_probs.pkl", "wb") as f:
    pickle.dump(pd.DataFrame(all_train_probs), f)
with open("all_test_probs.pkl", "wb") as f:
    pickle.dump(pd.DataFrame(all_test_probs), f)

In [ ]:
with open("F1_mats.json", "w") as f:
    f.write(json.dumps({m: F1_mats[m].tolist() for m in F1_mats}))
with open("lag_mats.json", "w") as f:
    f.write(json.dumps({m: lag_mats[m].tolist() for m in lag_mats}))

In [ ]:
[r for r in F1_mats if not np.any(np.isnan(F1_mats[r]))]

In [ ]:
consider_regions = {"milan", "hongkong", "barcelona", "wuhan", "manchester", "madrid"}

In [ ]:
# Show zeta vs time for each k x explained var in one plot. 
# Show latency vs k vs explained var. 
# Show F1 score (ignoring the k timesteps at the boundary) vs k and explained var

In [ ]:
matplotlib.rc('xtick', labelsize=18) 
matplotlib.rc('ytick', labelsize=18) 
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 18}
matplotlib.rc('font', **font)

In [ ]:
avg_F1s = np.nanmean([F1_mats[r] for r in consider_regions], axis=0)
plt.figure(dpi=1000, figsize=(7 * 3, 7 * 2))
plt.suptitle("$F_{1/6}$ score")
for a_idx in range(6):
    plt.subplot(2, 3, a_idx+1)
    plt.title(f"alpha={alphas[a_idx]}")
    plt.imshow(avg_F1s[:,:,a_idx], origin="lower", cmap="viridis", vmin=0, vmax=1)
    ax = plt.gca()
    ax.set_xticks(np.arange(len(k_values)), labels=k_values)
    ax.set_yticks(np.arange(len(explained_var_thresh_values)), labels=[round(v, 2) for v in explained_var_thresh_values])
    # Rotate the tick labels and set their alignment.
    # plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
    #          rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(explained_var_thresh_values)):
        for j in range(len(k_values)):
            text = ax.text(j, i, round(avg_F1s[i, j, a_idx], 2),
                        ha="center", va="center", color="w", weight="bold")
    plt.ylabel("explained variance")
    plt.xlabel("k")
plt.tight_layout()
                       

In [ ]:
for m in lag_mats.values():
    m[np.isnan(m) | (m == np.inf) | (m == 100) | (m == 100 * t_scale)] = np.nan

In [ ]:
t_scale = 24 * 60 * 60
avg_lags = np.nanmean([lag_mats[r] for r in consider_regions], axis=0) / t_scale
plt.figure(dpi=1000, figsize=(7 * 3, 7 * 2))
plt.suptitle("latency (days from lockdown), conditioned on detection")
for a_idx in range(6):
    plt.subplot(2, 3, a_idx+1)
    plt.title(f"alpha={alphas[a_idx]}")
    plt.imshow(avg_lags[:,:,a_idx], cmap="viridis_r", origin="lower", vmin=0, vmax=20)
    ax = plt.gca()
    ax.set_xticks(np.arange(len(k_values)), labels=k_values)
    ax.set_yticks(np.arange(len(explained_var_thresh_values)), labels=[round(v, 2) for v in explained_var_thresh_values])
    # Rotate the tick labels and set their alignment.
    # plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
    #          rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(explained_var_thresh_values)):
        for j in range(len(k_values)):
            text = ax.text(j, i, round(avg_lags[i, j, a_idx], 1),
                        ha="center", va="center", color="w", weight="bold")
    plt.ylabel("explained variance")
    plt.xlabel("k")
plt.tight_layout()
                       

### Calibration for multidim

In [ ]:
with open("all_test_ts.pkl", "rb") as f:
    all_test_ts = pickle.load(f).to_dict()
with open("all_test_probs.pkl", "rb") as f:
    all_test_probs = pickle.load(f).to_dict()

In [ ]:
lockdown_dates.keys()

In [ ]:
# seattle had an unusual snowstorm keeping people inside early jan (https://www.seattlepi.com/local/weather/article/Seattle-history-cold-arctic-ice-snow-storm-1950-14974927.php)
# madrid, barcelona has storm gloria
# tokyo, jerusalem, paris, losangeles, santiago not in all_test... (Exception)
# Birmingham, London, Berlin storm ciara https://www.trtworld.com/europe/uk-faces-another-fierce-storm-2-found-dead-in-rough-seas-33827, https://en.wikipedia.org/wiki/Storm_Ciara
consider_regions = {"milan", "hongkong", "newyork", "teipei", "manchester"}  
consider_test_ts = dict((r, all_test_ts[0.9][168][r]) for r in consider_regions)
consider_test_probs = dict((r, all_test_probs[0.9][168][r]) for r in consider_regions)
for region in consider_regions:
    lt = time.mktime(lockdown_dates[region].timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
    print(lt)
    idx = np.nonzero(consider_test_ts[region] < lt)[0].max()
    consider_test_probs[region] = consider_test_probs[region][:idx]
    consider_test_ts[region] = consider_test_ts[region][:idx]

In [ ]:
n_bins = 16
# bins = np.linspace(0, 1, n_bins + 1)
bins = 10**-np.linspace(0, 6, n_bins+1)[::-1]
pre_lockdown_counts = np.zeros(n_bins)
false_pos = 0
plt.figure(figsize=(8, 3))
n_samples = 0
for i, region in enumerate(consider_regions):
    pre_lockdown_counts += np.histogram(consider_test_probs[region], bins=bins, density=False)[0]
    false_pos += np.any(consider_test_probs[region] < alpha)
    n_samples += len(consider_test_probs[region])
    # plt.ylim([0, 10])
pre_lockdown_counts /= (bins[1:] - bins[:-1]) * n_samples
plt.subplot(1, 2, 1)
plt.xlim([10**-5, 1])
# plt.semilogy()
# plt.semilogx()
plt.bar(bins[:-1], pre_lockdown_counts, align="edge", label="pre-lockdown", color="m", alpha=0.4, width=bins[1:] - bins[:-1])
plt.legend(loc="upper left", bbox_to_anchor=[0, 1])
plt.semilogx()
plt.xlabel("p-value")
plt.ylabel("probability density")
plt.subplot(1, 2, 2)
n_bins = 10
bins = np.linspace(0, 1, n_bins + 1)
pre_lockdown_counts = np.zeros(n_bins)
n_samples = 0
for i, region in enumerate(consider_regions):
    pre_lockdown_counts += np.histogram(consider_test_probs[region], bins=bins, density=False)[0]
    false_pos += np.any(consider_test_probs[region] < alpha)
    n_samples += len(consider_test_probs[region])
    # plt.ylim([0, 10])
# plt.semilogy()
# plt.semilogx()
pre_lockdown_counts *= n_bins / n_samples
plt.bar(bins[:-1], pre_lockdown_counts, align="edge", label="pre-lockdown", color="m", alpha=0.4, width=bins[1:] - bins[:-1])
plt.xlabel("p-value")
plt.ylabel("probability density")
plt.tight_layout()
plt.savefig("validation_hist.svg", format="svg")
plt.show()

## Search for anomalies accounted for by the model in an individual station

In [ ]:
import matplotlib
import time
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 20}
matplotlib.rc('font', **font)

station_names = {'Station0004436',}  # {'Station0004349', 'Station0004436', 'Station0004701'}
explained_var_thresh = 0.8
k = 168
lamb = 2
alpha = 0.001

train_start_date = dt.datetime(2018, 1, 1)
train_end_date = dt.datetime(2020, 1, 1)
test_end_date = dt.datetime(2021, 1, 1)
t_min = time.mktime(dt.datetime(2018, 1, 1).timetuple())
train_start_t = time.mktime(train_start_date.timetuple()) - t_min
train_end_t = time.mktime(train_end_date.timetuple()) - t_min
test_end_t = time.mktime(test_end_date.timetuple()) - t_min

plt.figure(dpi=1000, figsize=(14, 9))
scale_t = 60 * 60 * 24
offset_t = 973
for i, station in enumerate(station_names):
    stations = [station]

    # load the koopman model and relevant data for station
    obs_t, obs_z_scores = load_z_scores(f"obs_{station}")

    # model_analys
    mod_t, mod_z_scores = load_z_scores(f"model_forecast_{station}")

    train_start = np.nonzero(obs_t >= train_start_t)[0].min()
    train_end = np.nonzero(obs_t >= train_end_t)[0].min()
    test_end = np.nonzero(obs_t <= test_end_t)[0].max()

    f = interpolate.interp1d(mod_t, mod_z_scores)
    aligned_mod_z_scores = f(obs_t)
    zeta_scores = obs_z_scores - aligned_mod_z_scores
    train_k_averages = get_contiguous_k_samples(k, zeta_scores[train_start:train_end]).mean(axis=1)
    obs_train_k_averages = get_contiguous_k_samples(k, obs_z_scores[train_start:train_end]).mean(axis=1)
    mod_train_k_averages = get_contiguous_k_samples(k, aligned_mod_z_scores[train_start:train_end]).mean(axis=1)
    train_ts = get_contiguous_k_samples(k, obs_t[train_start:train_end]).max(axis=1)  # represents the "real" time
    test_k_averages = get_contiguous_k_samples(k, zeta_scores[train_end:test_end]).mean(axis=1)
    obs_test_k_averages = get_contiguous_k_samples(k, obs_z_scores[train_end:test_end]).mean(axis=1)
    mod_test_k_averages = get_contiguous_k_samples(k, aligned_mod_z_scores[train_end:test_end]).mean(axis=1)
    test_ts = get_contiguous_k_samples(k, obs_t[train_end:test_end]).max(axis=1)

    # remove outliers from training data
    zeta_train_ts, train_k_averages = remove_outliers(train_ts, train_k_averages, lamb=lamb)
    obs_train_ts, obs_train_k_averages = remove_outliers(train_ts, obs_train_k_averages, lamb=lamb)
    mod_train_ts, mod_train_k_averages = remove_outliers(train_ts, mod_train_k_averages, lamb=lamb)

    of_factor = 1.4 # [1, 1.4, 1.3][i]  # adjust for overfitting
    zeta_norm = norm(train_k_averages.mean(), of_factor * train_k_averages.std())
    zeta_cdf = get_nd_cdf(n=1, radial_density=zeta_norm)
    obs_cdf = get_nd_cdf(n=1, radial_density=norm(obs_train_k_averages.mean(), of_factor * obs_train_k_averages.std()))
    mod_cdf = get_nd_cdf(n=1, radial_density=norm(mod_train_k_averages.mean(), of_factor * mod_train_k_averages.std()))
    train_probs, test_probs = (1 - zeta_cdf(abs(train_k_averages))), (1 - zeta_cdf(abs(test_k_averages)))
    zeta_crit_l, zeta_crit_u = zeta_norm.ppf(alpha), zeta_norm.mean() + (zeta_norm.mean() - zeta_norm.ppf(alpha))
    obs_train_probs, obs_test_probs = (1 - obs_cdf(abs(obs_train_k_averages))), (1 - obs_cdf(abs(obs_test_k_averages)))
    mod_train_probs, mod_test_probs = (1 - mod_cdf(abs(mod_train_k_averages))), (1 - mod_cdf(abs(mod_test_k_averages)))

    typeI_mask = (test_probs > alpha) & (obs_test_probs < alpha)
    if i < 9:
        plt.subplot(2, 1, 2*i + 1)
        plt.title("Seattle: " + station)
        lt = time.mktime(dt.datetime(2020, 9, 8).timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
        plt.fill_between([lt / scale_t - offset_t, lt / scale_t - offset_t + 35], [-11, -11], [11, 11], color="y", alpha=0.2, label="wildfires")
        plt.plot(test_ts / scale_t - offset_t, test_k_averages, color="c", linewidth=4, label="$\\bar \zeta$")
        plt.plot(test_ts / scale_t - offset_t, obs_test_k_averages, color="m", label="$\\bar z_{obs}$")
        plt.plot(test_ts / scale_t - offset_t, mod_test_k_averages, color="b", label="$\\bar z_{mod}$")
        plt.fill_between(test_ts / scale_t - offset_t, obs_test_k_averages, mod_test_k_averages, color="c",
                         alpha=0.5, linewidth=0)
        plt.axhline(zeta_crit_l, color="gray", linestyle="--")
        plt.axhline(0, color="gray", linestyle="-")
        plt.axhline(zeta_crit_u, color="gray", linestyle="--", label="$\\bar \zeta^*$")
        plt.fill_betweenx([-3, 3], [test_ts[typeI_mask][0] / scale_t - offset_t]*2, [test_ts[typeI_mask][-1] / scale_t - offset_t]*2, color=(1., 1., 1., 0.), edgecolor=(1., 0., 0., 0.2), hatch="xx", label="mitigated type I err")
        plt.gca().set_xlim(left=5, right=60)
        plt.ylim([-2.6, 2.6])
plt.xlabel("days since Sept 1 2020")
plt.ylabel(f"{k}-hr mean statistic")
plt.legend(loc=[1.04, 0.17]) # loc=[0, -1])

station_names = {'EEA_ES_ES1426A',}# {'EEA_IT_IT1251A','EEA_IT_IT1203A','EEA_IT_IT2232A'}
explained_var_thresh = 0.8
k = 168
lamb = 2
alpha = 0.001

train_start_date = dt.datetime(2018, 1, 1)
train_end_date = dt.datetime(2020, 1, 1)
test_end_date = dt.datetime(2021, 1, 1)
t_min = time.mktime(dt.datetime(2018, 1, 1).timetuple())
train_start_t = time.mktime(train_start_date.timetuple()) - t_min
train_end_t = time.mktime(train_end_date.timetuple()) - t_min
test_end_t = time.mktime(test_end_date.timetuple()) - t_min

scale_t = 60 * 60 * 24
offset_t = 789
for i, station in enumerate(station_names):
    stations = [station]

    # load the koopman model and relevant data for station
    obs_t, obs_z_scores = load_z_scores(f"obs_{station}")

    # model_analys
    mod_t, mod_z_scores = load_z_scores(f"model_forecast_{station}")

    train_start = np.nonzero(obs_t >= train_start_t)[0].min()
    train_end = np.nonzero(obs_t >= train_end_t)[0].min()
    test_end = np.nonzero(obs_t <= test_end_t)[0].max()

    f = interpolate.interp1d(mod_t, mod_z_scores)
    aligned_mod_z_scores = f(obs_t)
    zeta_scores = obs_z_scores - aligned_mod_z_scores
    train_k_averages = get_contiguous_k_samples(k, zeta_scores[train_start:train_end]).mean(axis=1)
    obs_train_k_averages = get_contiguous_k_samples(k, obs_z_scores[train_start:train_end]).mean(axis=1)
    mod_train_k_averages = get_contiguous_k_samples(k, aligned_mod_z_scores[train_start:train_end]).mean(axis=1)
    train_ts = get_contiguous_k_samples(k, obs_t[train_start:train_end]).max(axis=1)  # represents the "real" time
    test_k_averages = get_contiguous_k_samples(k, zeta_scores[train_end:test_end]).mean(axis=1)
    obs_test_k_averages = get_contiguous_k_samples(k, obs_z_scores[train_end:test_end]).mean(axis=1)
    mod_test_k_averages = get_contiguous_k_samples(k, aligned_mod_z_scores[train_end:test_end]).mean(axis=1)
    test_ts = get_contiguous_k_samples(k, obs_t[train_end:test_end]).max(axis=1)

    # remove outliers from training data
    zeta_train_ts, train_k_averages = remove_outliers(train_ts, train_k_averages, lamb=lamb)
    obs_train_ts, obs_train_k_averages = remove_outliers(train_ts, obs_train_k_averages, lamb=lamb)
    mod_train_ts, mod_train_k_averages = remove_outliers(train_ts, mod_train_k_averages, lamb=lamb)

    of_factor = 1
    zeta_norm = norm(train_k_averages.mean(), of_factor * train_k_averages.std())
    zeta_cdf = get_nd_cdf(n=1, radial_density=zeta_norm)
    obs_cdf = get_nd_cdf(n=1, radial_density=norm(obs_train_k_averages.mean(), of_factor * obs_train_k_averages.std()))
    mod_cdf = get_nd_cdf(n=1, radial_density=norm(mod_train_k_averages.mean(), of_factor * mod_train_k_averages.std()))
    train_probs, test_probs = (1 - zeta_cdf(abs(train_k_averages))), (1 - zeta_cdf(abs(test_k_averages)))
    zeta_crit_l, zeta_crit_u = zeta_norm.ppf(alpha), zeta_norm.mean() + (zeta_norm.mean() - zeta_norm.ppf(alpha))
    obs_train_probs, obs_test_probs = (1 - obs_cdf(abs(obs_train_k_averages))), (1 - obs_cdf(abs(obs_test_k_averages)))
    mod_train_probs, mod_test_probs = (1 - mod_cdf(abs(mod_train_k_averages))), (1 - mod_cdf(abs(mod_test_k_averages)))
    
    typeII_mask = (test_probs < alpha) & (obs_test_probs > alpha)
    if i < 9:
        plt.subplot(2, 1, 2*i + 2)
        plt.title("Madrid: " + station)
        plt.plot(test_ts / scale_t - offset_t, test_k_averages, color="c", linewidth=4)
        plt.plot(test_ts / scale_t - offset_t, obs_test_k_averages, color="m")
        plt.plot(test_ts / scale_t - offset_t, mod_test_k_averages, color="b")
        plt.fill_between(test_ts / scale_t - offset_t, obs_test_k_averages, mod_test_k_averages, color="c",
                         alpha=0.5, linewidth=0)
        lt = time.mktime(lockdown_dates["madrid"].timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
        plt.fill_between([lt / scale_t - offset_t, lt / scale_t - offset_t + 50], [-11, -11], [11, 11], color="y", alpha=0.2, label="lockdown")
        plt.axhline(zeta_crit_l, color="gray", linestyle="--")
        plt.axhline(0, color="gray", linestyle="-")
        plt.axhline(zeta_crit_u, color="gray", linestyle="--")
        plt.fill_betweenx([-3, 3], [test_ts[typeII_mask][0] / scale_t - offset_t]*2, [test_ts[typeII_mask][-1] / scale_t - offset_t]*2, color=(1., 1., 1., 0.), edgecolor=(1., 0., 0., 0.2), hatch="/", label="mitigated type II err")
        plt.gca().set_xlim(left=1, right=60)
        plt.ylim([-2.6, 2.6])
plt.xlabel("days since March 1 2020")
plt.legend(loc=[1.04, 0.72]) # loc=[0, -1])
plt.ylabel(f"{k}-hr mean statistic")
plt.tight_layout()
plt.savefig("seattle_v_madrid_zeta.pdf", format="pdf")
plt.show()

## z vs zeta figure

In [ ]:
import matplotlib
matplotlib.rc('xtick', labelsize=16) 
matplotlib.rc('ytick', labelsize=16) 
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 16}
matplotlib.rc('font', **font)

station = "EEA_ES_ES0118A"
explained_var_thresh = 0.8
k = 168
lamb = 2
alpha = 0.001

train_start_date = dt.datetime(2018, 1, 1)
train_end_date = dt.datetime(2020, 1, 1)
test_end_date = dt.datetime(2021, 1, 1)
t_min = time.mktime(dt.datetime(2018, 1, 1).timetuple())
train_start_t = time.mktime(train_start_date.timetuple()) - t_min
train_end_t = time.mktime(train_end_date.timetuple()) - t_min
test_end_t = time.mktime(test_end_date.timetuple()) - t_min

scale_t = 60 * 60 * 24
offset_t = 0
stations = [station]

# load the koopman model and relevant data for station
obs_t, obs_z_scores = load_z_scores(f"obs_{station}")
mod_t, mod_z_scores = load_z_scores(f"model_forecast_{station}")

train_start = np.nonzero(obs_t >= train_start_t)[0].min()
train_end = np.nonzero(obs_t >= train_end_t)[0].min()
test_end = np.nonzero(obs_t <= test_end_t)[0].max()

f = interpolate.interp1d(mod_t, mod_z_scores)
aligned_mod_z_scores = f(obs_t)
zeta_scores = obs_z_scores - aligned_mod_z_scores
train_k_averages = get_contiguous_k_samples(k, zeta_scores[train_start:train_end]).mean(axis=1)
obs_train_k_averages = get_contiguous_k_samples(k, obs_z_scores[train_start:train_end]).mean(axis=1)
mod_train_k_averages = get_contiguous_k_samples(k, aligned_mod_z_scores[train_start:train_end]).mean(axis=1)
train_ts = get_contiguous_k_samples(k, obs_t[train_start:train_end]).max(axis=1)  # represents the "real" time
test_k_averages = get_contiguous_k_samples(k, zeta_scores[train_end:test_end]).mean(axis=1)
obs_test_k_averages = get_contiguous_k_samples(k, obs_z_scores[train_end:test_end]).mean(axis=1)
mod_test_k_averages = get_contiguous_k_samples(k, aligned_mod_z_scores[train_end:test_end]).mean(axis=1)
test_ts = get_contiguous_k_samples(k, obs_t[train_end:test_end]).max(axis=1)

# remove outliers from training data
zeta_train_ts, zeta_train_k_averages = remove_outliers(train_ts, train_k_averages, lamb=lamb)
obs_train_ts, obs_train_k_averages = remove_outliers(train_ts, obs_train_k_averages, lamb=lamb)

of_factor = 1  # adjust for overfitting
obs_norm = norm(obs_train_k_averages.mean(), of_factor * obs_train_k_averages.std())
zeta_norm = norm(zeta_train_k_averages.mean(), of_factor * zeta_train_k_averages.std())
obs_cdf = get_nd_cdf(n=1, radial_density=obs_norm)
zeta_cdf = get_nd_cdf(n=1, radial_density=zeta_norm)
obs_train_probs, obs_test_probs = (1 - obs_cdf(abs(obs_train_k_averages))), (1 - obs_cdf(abs(obs_test_k_averages)))
zeta_train_probs, zeta_test_probs = (1 - zeta_cdf(abs(zeta_train_k_averages))), (1 - zeta_cdf(abs(test_k_averages)))
z_crit = -obs_norm.ppf(alpha)
zeta_crit = -zeta_norm.ppf(alpha)

layout = \
"""
AAB
"""
fig, (ax1, ax2) = plt.subplots(ncols=2, dpi=1500, figsize=(18, 4), gridspec_kw={'width_ratios': [2, 1]})

plt.sca(ax1)
lt = time.mktime(lockdown_dates["madrid"].timetuple()) - t_min
ax1.plot(obs_train_ts / scale_t - offset_t, obs_train_k_averages, color="c", label="$\\bar z_{obs}$")
ax1.plot(test_ts[test_ts < lt + 60 * scale_t] / scale_t - offset_t, obs_test_k_averages[test_ts < lt + 60 * scale_t], color="c")
plt.fill_between([0, train_end_t / scale_t], [-3, -3], [3, 3], color="c", alpha=0.2, label="train")
plt.fill_between([train_end_t / scale_t, lt / scale_t], [-3, -3], [3, 3], color="m", alpha=0.2, label="pre-lockdown")
plt.fill_between([lt / scale_t, lt / scale_t + 60], [-3, -3], [3, 3], color="y", alpha=0.2, label="lockdown")
plt.axhline(z_crit, color="gray", linestyle="--")
plt.axhline(-z_crit, label="$\\bar z^*$", color="gray", linestyle="--")
plt.axhline(0, color="gray", linestyle="-")
ax1.set_xlabel("days since Jan 1 2018")
ax1.set_ylabel("$\\bar z_{obs}$")
ax1.legend(loc=[1.04, 0.36]) # loc=[0, -1])
plt.ylim([-2.8, 2.8])

plt.sca(ax2)
plt.hist(obs_test_k_averages[test_ts < lt], color="m", alpha=0.4, label="pre-lockdown", density=True)
plt.hist(obs_test_k_averages[(lt < test_ts) & (test_ts < lt + 60 * scale_t)], color="y", alpha=0.4, label="lockdown", density=True)
plt.hist(obs_train_k_averages, color="c", alpha=0.4, label="train", density=True)
plt.axvline(z_crit, label="$\\bar z^*$", color="gray", linestyle="--")
plt.axvline(-z_crit, color="gray", linestyle="--")
plt.axvline(0, color="gray", linestyle="-")
ax2.set_ylabel("density")
ax2.set_xlabel("$\\bar z_{obs}$")
plt.ylim([0, 2.7])
plt.xlim([-2.8, 2.8])

plt.tight_layout()
plt.savefig("z_" + station + ".pdf", format="pdf")
plt.show()


In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, dpi=1500, figsize=(18, 4), gridspec_kw={'width_ratios': [2, 1]})

plt.sca(ax1)
ax1.plot(train_ts / scale_t - offset_t, train_k_averages, color="c", label="$\\bar \zeta$")
ax1.plot(test_ts[test_ts < lt + 60 * scale_t] / scale_t - offset_t, test_k_averages[test_ts < lt + 60 * scale_t], color="c")
lt = time.mktime(lockdown_dates["madrid"].timetuple()) - t_min
plt.fill_between([0, train_end_t / scale_t], [-3, -3], [3, 3], color="c", alpha=0.2, label="train")
plt.fill_between([train_end_t / scale_t, lt / scale_t], [-3, -3], [3, 3], color="m", alpha=0.2, label="pre-lockdown")
plt.fill_between([lt / scale_t, lt / scale_t + 60], [-3, -3], [3, 3], color="y", alpha=0.2, label="lockdown")
plt.axhline(zeta_crit, color="gray", linestyle="--")
plt.axhline(-zeta_crit, label="$\\bar \zeta^*$", color="gray", linestyle="--")
plt.axhline(0, color="gray", linestyle="-")
ax1.set_xlabel("days since Jan 1 2018")
ax1.set_ylabel("$\\bar \zeta$")
ax1.legend(loc=[1.04, 0.36]) # loc=[0, -1])
plt.ylim([-2.8, 2.8])

plt.sca(ax2)
plt.hist(test_k_averages[test_ts < lt], color="m", alpha=0.4, label="pre-lockdown", density=True)
plt.hist(test_k_averages[(lt < test_ts) & (test_ts < lt + 60 * scale_t)], color="y", alpha=0.4, label="lockdown", density=True)
plt.hist(train_k_averages, color="c", alpha=0.4, label="train", density=True)
plt.axvline(zeta_crit, label="$\\bar \zeta^*$", color="gray", linestyle="--")
plt.axvline(-zeta_crit, color="gray", linestyle="--")
plt.axvline(0, color="gray", linestyle="-")
ax2.set_ylabel("density")
ax2.set_xlabel("$\\bar \zeta$")
plt.ylim([0, 2.7])
plt.xlim([-2.8, 2.8])

plt.tight_layout()
plt.savefig("zeta_" + station + ".pdf", format="pdf")
plt.show()

## Global Map figure

In [ ]:
alpha=0.001
k=168
quality_thresh=0.5
explained_var_thresh=0.9

country1_region = "madrid-barcelona"
country1_stations = get_stations(country1_region, quality_thresh=quality_thresh)
country1_train_ts, country1_test_ts, country1_train_zeta_avg_mat, country1_test_zeta_avg_mat, train_end_t = get_zeta_series(country1_stations, k=k, show=False)
train_mahalanobis_dist, test_mahalanobis_dist, num_PCs = get_mahalanobis_dist(country1_train_ts, country1_test_ts,
            country1_train_zeta_avg_mat, country1_test_zeta_avg_mat, train_end_t, explained_var_thresh=explained_var_thresh, show=False)
cdf = get_nd_cdf(n=num_PCs, radial_density=norm)
country1_train_probs, country1_test_probs = (1 - cdf(abs(train_mahalanobis_dist))), (1 - cdf(abs(test_mahalanobis_dist)))

In [ ]:
# China + Hong Kong
alpha=0.001
k=168
quality_thresh=0.5
explained_var_thresh=0.9

country2_region = "beijing-shanghai-wuhan"
country2_stations = get_stations(country2_region, quality_thresh=quality_thresh)
country2_train_ts, country2_test_ts, country2_train_zeta_avg_mat, country2_test_zeta_avg_mat, train_end_t = get_zeta_series(country2_stations, k=k, show=False)
train_mahalanobis_dist, test_mahalanobis_dist, num_PCs = get_mahalanobis_dist(country2_train_ts, country2_test_ts,
            country2_train_zeta_avg_mat, country2_test_zeta_avg_mat, train_end_t, explained_var_thresh=explained_var_thresh, show=False)
cdf = get_nd_cdf(n=num_PCs, radial_density=norm)
country2_train_probs, country2_test_probs = (1 - cdf(abs(train_mahalanobis_dist))), (1 - cdf(abs(test_mahalanobis_dist)))

In [ ]:
from PIL import Image
im = Image.open("SEDAC_POP_2000-01-01_rgb_1440x720.TIFF")
world = np.array(im)

In [ ]:
import matplotlib.colors
import matplotlib 

pad = 4
bworld = np.zeros([world.shape[0] + pad*2, world.shape[1] + pad*2])
bworld[pad:-pad, pad:-pad] = world

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

with open("data/metadata.json") as f:
        metadata = json.loads(f.read())

temp = []
for fname in metadata:
    temp.append(metadata[fname])
    temp[-1]["fname"] = fname
df_meta = pd.DataFrame(temp)

In [ ]:
# # Load analysis for major cities

# with open("all_test_ts.pkl", "rb") as f:
#     all_test_ts = pickle.load(f).to_dict()
# with open("all_test_probs.pkl", "rb") as f:
#     all_test_probs = pickle.load(f).to_dict()

In [ ]:
matplotlib.rc('xtick', labelsize=18) 
matplotlib.rc('ytick', labelsize=18) 
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 18}
matplotlib.rc('font', **font)

In [ ]:
prob_cmap = truncate_colormap(plt.get_cmap('plasma'), 0, 0.95)
prob_vmin = -5
prob_vmax = 0
snap1_date = dt.datetime(2020, 2, 1)
snap2_date = dt.datetime(2020, 5, 1)
snap1t = time.mktime(snap1_date.timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
snap2t = time.mktime(snap2_date.timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())

In [ ]:
t_scale = 24* 60 * 60
offset = 365 * 2
plt.figure(figsize=(20, 3), dpi=200)

# storm gloria in spain Jan 20
gloria_t = time.mktime(dt.datetime(2020, 1, 19).timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
gloria_end_t = time.mktime(dt.datetime(2020, 1, 26).timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
plt.fill_between([gloria_t / t_scale - offset, gloria_end_t / t_scale - offset], [0, 0], [11, 11], color="b", alpha=0.2, label="Storm Gloria")
lt = time.mktime(lockdown_dates["barcelona"].timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
plt.fill_between([lt / t_scale - offset, lt / t_scale - offset + 100], [0, 0], [11, 11], color="y", alpha=0.2, label="lockdown")
plt.axvline(snap1t / t_scale - offset, linestyle="-", color="gray")
plt.axvline(snap2t / t_scale - offset, linestyle="-", color="gray")
colors="cmyrgb"
i=0
plt.plot(country1_test_ts / t_scale - offset, country1_test_probs, linewidth=2, color="c")
plt.axhline(alpha, color="gray", linestyle="--", label="$\\alpha$")
# plt.ylim([10**-6, 1])
plt.xlim([-10, 135])
plt.semilogy()
plt.legend()
plt.xlabel("days since Jan 1 2020")
plt.ylabel("p-value")

# for i, region in enumerate(test_ts):
#     # _tr_t = np.log10(all_train_ts[region])
#     # _te_t = np.log10(all_test_ts[region])
#     # tr_t = 360 * (_tr_t - _tr_t.min()) / (_te_t.max() - _tr_t.min()) - 180
#     # te_t = 360 * (_te_t - _tr_t.min()) / (_te_t.max() - _tr_t.min()) - 180
#     # _tr_p = np.log10(train_probs[region])
#     # _te_p = np.log10(test_probs[region])
#     # tr_p = 30 * (_tr_p - _te_p.min()) / (_tr_p.max() - _te_p.min()) + 90
#     # te_p = 30 * (_te_p - _te_p.min()) / (_tr_p.max() - _te_p.min()) + 90
#     tr_p = train_probs[region]
#     te_p = test_probs[region]
#     tr_t = all_train_ts[region]
#     te_t = all_test_ts[region]
#     scale_t = 60 * 60 * 24
#     ax1.plot(tr_t / scale_t, tr_p, color=colors[i], label=city_names[region])
#     ax1.plot(te_t / scale_t, te_p, color=colors[i])
#     lt = time.mktime(lockdown_dates[region].timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
#     ax1.axvline(lt / scale_t, color=colors[i], linestyle="--")
#     ax1.semilogy()
#     ax1.set_ylabel("p-value")
#     ax1.set_xlabel("days since Jan 1 2018")
# ax1.axvline(365 * 2, color="gray")
plt.legend(loc="lower left", bbox_to_anchor=[0, 0])
plt.gca().set_ylim(top=3)
plt.title("Spain")
plt.savefig("spain_p.svg", format="svg")
plt.show()

In [ ]:
t_scale = 24* 60 * 60
offset = 365 * 2
plt.figure(figsize=(20, 3), dpi=200)
lt = time.mktime(lockdown_dates["wuhan"].timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
lt_end = time.mktime(dt.datetime(2020, 3, 22).timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
plt.fill_between([lt / t_scale - offset, lt_end / t_scale - offset], [0, 0], [11, 11], color="y", alpha=0.2, label="Wuhan lockdown")
chinese_ny_start_date = dt.datetime(2020, 1, 25)
cnyt = time.mktime(chinese_ny_start_date.timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
plt.fill_between([cnyt / t_scale - offset, cnyt / t_scale - offset + 15], [0, 0], [11, 11], color="red", alpha=0.2, label="Chinese New Year")
plt.axvline(snap1t / t_scale - offset, linestyle="-", color="gray")
plt.axvline(snap2t / t_scale - offset, linestyle="-", color="gray")

plt.plot(country2_test_ts / t_scale - offset, country2_test_probs, linewidth=2, color="c")
plt.axhline(alpha, color="gray", linestyle="--", label="$\\alpha$")
plt.xlim([-10, 135])
plt.semilogy()
plt.legend()
plt.xlabel("days since Jan 1 2020")
plt.ylabel("p-value")

plt.title("China")
plt.legend(loc="lower left", bbox_to_anchor=[0, 0])
plt.gca().set_ylim(top=3)
plt.savefig("china_p.svg", format="svg")
plt.show()

In [ ]:
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 25}
matplotlib.rc('font', **font)

# snapshot_dates = [dt.datetime(2020, 2, 1), dt.datetime(2020, 3, 20), dt.datetime(2020, 5, 1)]
snapshot_date = snap1_date
snapshot_t = time.mktime(snapshot_date.timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
test_ts = all_test_ts[0.9][168]  # dict of region: test_ts
test_probs = all_test_probs[0.9][168]  # dict of region: test_probs
city_locs = []
city_probs = []
for region in test_ts:
    idxs = np.nonzero(test_ts[region] >= snapshot_t)[0]
    if len(idxs) != 0:
        idx = idxs.min()
    else:
        continue
    city_probs.append(test_probs[region][idx])
    city_locs.append(region_centers[region])
city_locs = np.array(city_locs)

plt.figure(dpi=1000, figsize=(20, 15))
plt.imshow(bworld, cmap=truncate_colormap(plt.get_cmap('bone'), 0.4, 0.75), vmin=0, vmax=255, extent=(-180, 180, -90, 90), aspect="equal")
city_stations = df_meta[(df_meta["quality"] > quality_thresh) & (df_meta["mean"] > obs_thresh) & (df_meta["obs"])]
locs = np.array([[df_meta["lat"].values, df_meta["lon"].values]])
plt.scatter(locs[:, 1], locs[:, 0], s=0.4, c="white", alpha=0.4)
sc = plt.scatter(city_locs[:, 1], city_locs[:, 0], s=120, c=np.log10(city_probs), alpha=1, cmap=prob_cmap, vmin=prob_vmin, vmax=prob_vmax)
plt.axis('off')
# plt.subplots_adjust(hspace=-0.5)
plt.tight_layout()
plt.colorbar(sc, label="$\log_{10}(p)$", shrink=0.5, location="right")
plt.savefig("world_map" + str(snapshot_date)[:10] + ".svg", format="svg")
plt.show()


In [ ]:
# white dot for each station, colored dots for a few cities in which we've done the analysis, plot time-series for hierarchy


layout = \
"""
AAAA
BBBB
CCCC
CCCC
"""
fig, axs = plt.subplot_mosaic(layout, dpi=1500, figsize=(15, 10))
axnames = "ABC"
snapshot_dates = [dt.datetime(2020, 2, 1), dt.datetime(2020, 3, 20), dt.datetime(2020, 5, 1)]
for i, snapshot_date in enumerate(snapshot_dates):
    snapshot_t = time.mktime(snapshot_date.timetuple()) - time.mktime(dt.datetime(2018, 1, 1).timetuple())
    test_ts = all_test_ts[0.9][168]  # dict of region: test_ts
    test_probs = all_test_probs[0.9][168]  # dict of region: test_probs
    city_locs = []
    city_probs = []
    for region in test_ts:
        idxs = np.nonzero(test_ts[region] >= snapshot_t)[0]
        if len(idxs) != 0:
            idx = idxs.min()
        else:
            continue
        city_probs.append(test_probs[region][idx])
        city_locs.append(region_centers[region])
    city_locs = np.array(city_locs)
    
    plt.sca(axs[axnames[i]])
    if i < len(snapshot_dates) - 1:
        plt.imshow(world[:world.shape[0] // 2, :], cmap=truncate_colormap(plt.get_cmap('bone'), 0.4, 0.75), vmin=0, vmax=255, extent=(-180, 180, 0, 90), aspect="equal")
    else:
        plt.imshow(world, cmap=truncate_colormap(plt.get_cmap('bone'), 0.4, 0.75), vmin=0, vmax=255, extent=(-180, 180, -90, 90), aspect="equal")
    city_stations = df_meta[(df_meta["quality"] > quality_thresh) & (df_meta["mean"] > obs_thresh) & (df_meta["obs"])]
    locs = np.array([[df_meta["lat"].values, df_meta["lon"].values]])
    plt.scatter(locs[:, 1], locs[:, 0], s=0.2, c="white", alpha=0.4)
    sc = plt.scatter(city_locs[:, 1], city_locs[:, 0], s=50, c=np.log10(city_probs), alpha=1, cmap=truncate_colormap(plt.get_cmap('plasma'), 0, 0.9), vmin=-5, vmax=0)
    plt.axis('off')
# plt.subplots_adjust(hspace=-0.5)
plt.tight_layout()
plt.colorbar(sc, ax=[ax for ax in axs.values()], label="$\log_{10}(p)$", shrink=0.6, location="right")
plt.savefig("world_map" + ",".join([str(s)[:10] for s in snapshot_dates]) + ".svg", format="svg")
plt.show()
